In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext()

In [3]:
raw_data = sc.textFile('/data/twitter/twitter_sample.txt')

In [4]:
def parse_edge(s):
    user, follower = s.split('\t')
    return (int(user), int(follower))

In [5]:
edges = raw_data.map(parse_edge).cache()

In [6]:
edges.take(5)

[(12, 18), (12, 41), (12, 57), (12, 62), (12, 235)]

### Find users with the largest number of followers

In [7]:
follower_counts = edges.mapValues(lambda v: 1).reduceByKey(lambda x, y: x + y)

In [8]:
follower_counts.top(5, key=lambda x: x[1])

[(20, 121425), (13, 102854), (12, 100113), (586, 92777), (291, 92026)]

In [9]:
# alternatively
import operator
follower_counts = edges.aggregateByKey(0, lambda a, x: a + 1, operator.iadd)
follower_counts.top(5, operator.itemgetter(1))

[(20, 121425), (13, 102854), (12, 100113), (586, 92777), (291, 92026)]

In [10]:
follower_counts.top(5, lambda x: -x[1])

[(518, 1), (540, 1), (32, 1), (612, 1), (700, 1)]

### Computing distances of all other vertices w.r.t. vertex - 12

In [11]:
n = 400  # number of partitions
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

In [12]:
connected_vertices = forward_edges.map(lambda edge: (edge[0], [edge[1]])).reduceByKey(lambda x, y: x + y)

In [13]:
connected_vertices.take(10)

[(37376000, [13, 20]),
 (15078400, [607]),
 (52339600, [12]),
 (46910800, [586]),
 (30238400, [586]),
 (48007200, [13]),
 (52024400, [12]),
 (15768800, [13]),
 (17878800, [12, 531]),
 (58756400, [13])]

In [14]:
connected_vertices.lookup(107)

[[15, 20, 586]]

In [15]:
start_node = 12
end_node = 34

In [16]:
parent = {}
queue = [start_node]
stop = False
while queue and not stop:
    current_node = queue.pop()
#     print(current_node)
    try:
        for vertex in connected_vertices.lookup(current_node)[0]:
    #         print(vertex)
            if not vertex in parent:
                # build path from child to parent
                parent[vertex] = current_node
                queue.append(vertex)
            if vertex == end_node:
                stop = True
                break
    except IndexError as e:
        continue

In [ ]:
path


In [17]:
path = [end_node]
while parent[end_node] is not None:
    path.append(parent[end_node])

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 35256)
----------------------------------------


Traceback (most recent call last):
  File "/usr/lib/python2.7/SocketServer.py", line 290, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python2.7/SocketServer.py", line 318, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python2.7/SocketServer.py", line 331, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python2.7/SocketServer.py", line 652, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pyspark/serializers.py", line 577, in read_int
    raise EOFError
EOFError
Exception KeyboardInterrupt in <function <lambda> at 0x7f0f1805c320> ignored
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 883, in send_command
    res

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34969)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34969)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34969)
Traceback (most recent call last):
  File "/usr

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34969)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34969)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34969)
Traceback (most recent call last):
  File "/usr

KeyboardInterrupt: 

In [34]:
connected_vertices.filter(lambda t: t[1] == []).collect()

[]

In [27]:
for vertex in connected_vertices.lookup(12):
    print(vertex)

[126, 380, 422, 648]


In [40]:
connected_vertices.lookup(380)

[[31, 690]]

In [11]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

def step(item):
    prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
    return (next_v, prev_d + 1)

def complete(item):
    v, old_d, new_d = item[0], item[1][0], item[1][1]
    return (v, old_d if old_d is not None else new_d)

n = 400  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

In [12]:
forward_edges.take(10)

[(16447200, 12),
 (16929600, 12),
 (22550800, 12),
 (22829200, 12),
 (23675600, 12),
 (25820800, 12),
 (25927600, 12),
 (32510400, 12),
 (33032400, 12),
 (35480400, 12)]

In [13]:
x = 12
d = 0
distances = sc.parallelize([(x, d)]).partitionBy(n)
while True:
    candidates = distances.join(forward_edges, n).map(step)
    new_distances = distances.fullOuterJoin(candidates, n).map(complete, True).persist()
    count = new_distances.filter(lambda i: i[1] == d + 1).count()
    if count > 0:
        d += 1
        distances = new_distances
    else:
        break

In [27]:
distances.lookup(34)

[8]

### Find the shortest path from 12 to 34

In [29]:
from __future__ import print_function
print(12, end='')
target = 34
dist = distances.lookup(target)[0]
while dist:
    

12

In [30]:
distances.filter(lambda x: x[1] == 1).collect()

[(422, 1), (126, 1), (648, 1), (380, 1)]

In [33]:
distances.filter(lambda x: x[1] == 4).collect()

[(13, 4),
 (107, 4),
 (107, 4),
 (107, 4),
 (107, 4),
 (107, 4),
 (107, 4),
 (107, 4),
 (107, 4),
 (107, 4),
 (107, 4),
 (107, 4),
 (107, 4)]